In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import nltk
import lyricsgenius
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm
import pickle
from scipy.stats.stats import pearsonr
from tqdm import tqdm
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from scipy.stats import pearsonr
from sklearn.cluster import KMeans

## Datasets

In [23]:
df_original = pd.read_pickle("lib/playlists.pkl")
df = df_original.explode("tracks")
df_track = pd.read_pickle("lib/tracks.pkl")
df_track_features = pd.read_csv("lib/tracks_features.csv", index_col=False)
df_clusters = pd.read_csv('lib/df_clusters.csv')

In [24]:
#Join playlist and tracks
df_track_features.rename(columns={'id': 'tracks'}, inplace=True)
df_merged = pd.merge(df_track_features, df, on='tracks',  how='left')
df_merged = df_merged.drop(['Unnamed: 0', 'pos', 'artist_name', 'track_uri', 'artist_uri',
       'track_name', 'album_uri', 'duration_ms_x', 'album_name', 'sparse_id', 
       'tracks', 'uri', 'track_href', 'analysis_url', 'duration_ms.1',
       'time_signature', 'name', 'collaborative', 'modified_at',
       'num_tracks', 'num_albums', 'num_followers', 'num_edits',
       'duration_ms_y', 'num_artists', 'description', 'type'], axis=1)
#One hot encoding for key
df_merged = pd.get_dummies(df_merged, columns=['key'])

In [152]:
df_merged.head()

,tid,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0UaMYEvWZi0ZqiDOoHU3YI,0.904,0.813,-7.105,0,0.121,0.0311,0.00697,0.0471,0.81,...,0,0,1,0,0,0,0,0,0,0
1,0UaMYEvWZi0ZqiDOoHU3YI,0.904,0.813,-7.105,0,0.121,0.0311,0.00697,0.0471,0.81,...,0,0,1,0,0,0,0,0,0,0
2,0UaMYEvWZi0ZqiDOoHU3YI,0.904,0.813,-7.105,0,0.121,0.0311,0.00697,0.0471,0.81,...,0,0,1,0,0,0,0,0,0,0
3,0UaMYEvWZi0ZqiDOoHU3YI,0.904,0.813,-7.105,0,0.121,0.0311,0.00697,0.0471,0.81,...,0,0,1,0,0,0,0,0,0,0
4,0UaMYEvWZi0ZqiDOoHU3YI,0.904,0.813,-7.105,0,0.121,0.0311,0.00697,0.0471,0.81,...,0,0,1,0,0,0,0,0,0,0


## Similarity Measures

In [3]:
def cosine_sim(df, pid, pids, k):
    cosines = []
    for i in range(len(df)):
        if i==pid:
            cosines.append((pids[i], 0))
            continue
        similarity = np.dot(df.iloc[pid], df.iloc[i])/(np.linalg.norm(df.iloc[pid])*np.linalg.norm(df.iloc[i]))
        cosines.append((pids[i], similarity))

    cosines.sort(key=lambda a: a[1], reverse=True)
    pid_similars = list(zip(*cosines[:k]))[0]
    similarity_values = list(zip(*cosines[:k]))[1]
    
    return pid_similars, similarity_values

In [4]:
def pearson_sim(df, pid, pids, k):
    pearsons = []
    for i in range(len(df)):
        if i==pid:
            pearsons.append((pids[i], 0))
            continue
        similarity = pearsonr(df.iloc[pid], df.iloc[i])[0]
        pearsons.append((pids[i], similarity))
        
    pearsons.sort(key=lambda a: a[1], reverse=True)
    pid_similars = list(zip(*pearsons[:k]))[0]
    similarity_values = list(zip(*pearsons[:k]))[1]
    
    return pid_similars, similarity_values

In [5]:
def euclidean_dist(df, pid, pids, k):
    dists = []
    for i in range(len(df)):
        similarity = np.linalg.norm(df.iloc[pid]-df.iloc[i])
        dists.append((pids[i], similarity))
        
    dists.sort(key=lambda a: a[1])
    pid_similars = list(zip(*dists[1:k+1]))[0]
    similarity_values = list(zip(*dists[1:k+1]))[1]
    
    return pid_similars, similarity_values

# K-Means

In [20]:
df_track_features.drop(columns=['time_signature','duration_ms','energy','duration_ms.1','track_uri','pos','artist_uri','album_uri','sparse_id','id','type','uri','track_href','analysis_url'],inplace=True)

def kmeans(df):
    # scale data -
    to_scale = ['danceability','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','positive','negative','neutral','compound']
    scaler = MinMaxScaler()
    df[to_scale] = scaler.fit_transform(df[to_scale])

    # OHE
    to_encode =['key']
    df = pd.get_dummies(df, columns=to_encode)
    df.head()

    # KMeans
    features = df.drop(['tid','track_name', 'artist_name','album_name'], axis=1)
    # Define the range of cluster numbers to try
    k_range = range(1, 11)

    # Calculate the within-cluster sum of squares for each value of k
    wcss = []
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(features)
        wcss.append(kmeans.inertia_)

    # Plot the elbow plot
    plt.plot(k_range, wcss, marker='o')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
    plt.title('Elbow Plot')
    plt.show()

    # Train the KMeans model
    num_clusters = 2
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(features)

    # Get the predicted cluster labels for each data point
    labels = kmeans.predict(features)

    # Add the predicted labels as a new column in the dataset
    df['cluster'] = labels

    return df

In [21]:
df_clusters = kmeans(df_track_features)

# Content-Based Filtering

In [19]:
def content_based_filtering(df,track,artist,n):
    
    dfc = df.copy()
    
    # filter out tracks in the same cluster as query
    cluster = dfc.loc[(dfc['track_name']==track) & (dfc['artist_name']==artist)]['cluster'].values[0]
    dfc = dfc[dfc['cluster']==cluster]
    
    # add one for query artist and album
    dfc['artist_flag'] = 0
    dfc.loc[dfc['artist_name'] == artist, 'artist_flag'] = 1
    dfc['album_flag'] = 0
    album = dfc.loc[dfc['track_name'] == track, 'album_name'].values[0]
    dfc.loc[dfc['album_name'] == album, 'album_flag'] = 1
    
    other_features = np.array(dfc.iloc[:,4:].values)
    song_feature = np.array(dfc.loc[(dfc['track_name']==track) & (dfc['artist_name']==artist)].iloc[:,4:].values).reshape(1,-1)
    
    #if metric == 'cosine':
    dfc['cosine_similarity'] = cosine_similarity(other_features, song_feature)
    top_cosine = dfc.sort_values('cosine_similarity',ascending = False).head(n+1)[1:][['artist_name', 'track_name', 'album_name','cosine_similarity']]

    #elif metric == 'euclidean':
    dfc['euclidean'] = pairwise_distances(other_features, song_feature, metric='euclidean')
    top_euclidean = dfc.sort_values('euclidean',ascending = True).head(n+1)[1:][['artist_name', 'track_name', 'album_name','euclidean']]

    #elif metric == 'pearson':
    dfc['pearson']= [pearsonr(song_feature.reshape(-1), x.reshape(-1))[0] for x in other_features]
    top_pearson = dfc.sort_values('pearson',ascending = False).head(n+1)[1:][['artist_name', 'track_name', 'album_name','pearson']]

    return top_cosine, top_euclidean, top_pearson

In [25]:
top_cosine, top_euclidean, top_pearson = content_based_filtering(df_clusters, 
                                                                 'I Want It That Way', 'Backstreet Boys', 5)

### Cosine Similarity

In [26]:
top_cosine

,artist_name,track_name,album_name,cosine_similarity
22993,Backstreet Boys,Show Me the Meaning of Being Lonely,Original Album Classics,0.989953
5527,Backstreet Boys,Everybody (Backstreet's Back) - Extended Version,Original Album Classics,0.853605
12618,Backstreet Boys,Larger Than Life,Original Album Classics,0.850272
10181,Nicky Jam,With You Tonight (Hasta El Amanecer),Fénix,0.849867
3709,Nick Jonas,Close,Last Year Was Complicated,0.849325


### Pearson Similarity

In [28]:
top_pearson

,artist_name,track_name,album_name,pearson
22993,Backstreet Boys,Show Me the Meaning of Being Lonely,Original Album Classics,0.983485
4643,Bill Medley,(I've Had) The Time of My Life,Platinum & Gold Collection,0.773162
13712,Ro James,Permission,ELDORADO,0.772849
1213,Halsey,Now Or Never,hopeless fountain kingdom,0.772678
2400,Major Lazer,Cold Water (feat. Justin Bieber & MØ),Cold Water (feat. Justin Bieber & MØ),0.772111


### Euclidean Distance

In [27]:
top_euclidean

,artist_name,track_name,album_name,euclidean
22993,Backstreet Boys,Show Me the Meaning of Being Lonely,Original Album Classics,0.399599
10181,Nicky Jam,With You Tonight (Hasta El Amanecer),Fénix,1.423700
3709,Nick Jonas,Close,Last Year Was Complicated,1.424058
14191,Josef Salvat,Open Season - Gryffin Remix,Open Season,1.426394
1213,Halsey,Now Or Never,hopeless fountain kingdom,1.427379


# Collaborative Filtering

In [7]:
def collaborative_filtering(df, df_track, df_original, pid, test_perc, k, similarity_measure): 

    #Remove test set
    pid_indices = df[df['pid']==pid].index
    total_size = len(pid_indices)
    test_size = int(total_size*(test_perc/100))

    test_tracks = []
    for index in pid_indices[:test_size]:
        test_tracks.append(df.loc[index]['tracks'])
        df = df.drop(index)

    #Remove unnecessary columns
    df = df.drop(['tid'], axis=1)


    #Get means
    df_means = df.groupby(['pid']).mean()
    pids = df_means.index.tolist()


    #Scaling
    scaler = MinMaxScaler()
    df_means = pd.DataFrame(scaler.fit_transform(df_means))
    df_means.index = pids

    if similarity_measure == 'cosine similarity':
        pid_similars, similarity_values = cosine_sim(df_means, pid, pids, k)
        
        
    elif similarity_measure == 'pearson similarity':
        pid_similars, similarity_values = pearson_sim(df_means, pid, pids, k)
        
    elif similarity_measure == 'euclidean distance':
        pid_similars, similarity_values = euclidean_dist(df_means, pid, pids, k)

    
    #Get original playlist tracks
    original_playlist = df_original[df_original['pid']==pid].iloc[0]['tracks']

    #Remove overlap with query playlist
    similar_playlists = []
    for pid_similar in pid_similars:
        similar_playlist = df_original[df_original['pid']==pid_similar].iloc[0]['tracks']
        overlap = set(original_playlist).intersection(similar_playlist)
        for track in overlap:
            similar_playlist.remove(track)
        similar_playlists.append(similar_playlist)

    similar_playlists =  [item for sublist in similar_playlists for item in sublist]
    counter = Counter(similar_playlists)
    intersection = [word for word,cnt in counter.most_common(5)]
    
    tracks = []
    tracks_count = [cnt for word,cnt in counter.most_common(5)]
    artists = []
    for track in intersection:
        tracks.append(df_track.loc[track]['track_name'])
        artists.append(df_track.loc[track]['artist_name'])

    df_similar_tracks = pd.DataFrame(list(zip(tracks, artists, tracks_count)), 
                                     columns =['Track name', 'Artist', 'No. of Playlists'])
    
    playlist_names = []
    for playlist in pid_similars:
        playlist_names.append(df_original.loc[playlist]['name'])
    
    df_similar_playlists = pd.DataFrame(list(zip(playlist_names, similarity_values)), 
                                        columns =['Playlist name', similarity_measure])
    
    
    return df_similar_tracks, df_similar_playlists, intersection, df_means.iloc[pid]

### Cosine Similarity

In [285]:
pid = 235
df_similar_tracks_1, df_similar_playlists_1, intersection_1, playlist_means_1 = collaborative_filtering(
                                                                                    df_merged, df_track, 
                                                                                      df_original, pid, 
                                                                                        0, 50, 
                                                                                      'cosine similarity')

In [286]:
print("Cosine Similarity")
df_similar_tracks_1

Cosine Similarity


,Track name,Artist,No. of Playlists
0,I Write Sins Not Tragedies,Panic! At The Disco,22
1,Centuries,Fall Out Boy,19
2,Welcome to the Black Parade,My Chemical Romance,17
3,My Songs Know What You Did In The Dark (Light ...,Fall Out Boy,17
4,Teenagers,My Chemical Romance,16


In [294]:
df_similar_playlists_1.head()

,Playlist name,cosine similarity
0,🎧🎧🎧,0.997311
1,My Songs,0.996865
2,Good old times,0.996589
3,Summer hits,0.996295
4,Jacob,0.996239


In [245]:
#Evaluation
df_predicted = pd.DataFrame()
for track in intersection_1:
    df_predicted = df_predicted.append(df_merged.loc[df_merged['tid'] == track].head(1))

df_random = pd.DataFrame()
random.seed(50)
for idx in random.sample(range(0, len(df_merged)), 10):
    df_random = df_random.append(df_merged.iloc[idx])
    
df_predicted = df_predicted.drop(['tid', 'pid'], axis=1)
df_random = df_random.drop(['tid', 'pid'], axis=1)

#Scale
scaler = MinMaxScaler()
df_predicted = pd.DataFrame(scaler.fit_transform(df_predicted))
df_random = pd.DataFrame(scaler.fit_transform(df_random))

#Get mean
df_predicted_means = df_predicted.mean()
df_random_means = df_random.mean()

#Similarity measure
print("Cosine similarity of predicted tracks:", 
      np.dot(df_predicted_means, playlist_means_1)/(np.linalg.norm(df_predicted_means)*np.linalg.norm(playlist_means_1)))
print("Cosine similarity of random tracks:", 
      np.dot(df_random_means, playlist_means_1)/(np.linalg.norm(df_random_means)*np.linalg.norm(playlist_means_1)))

Cosine similarity of predicted tracks: 0.9308948118242059
Cosine similarity of random tracks: 0.8935333069241516


### Pearson Similarity

In [288]:
df_similar_tracks_2, df_similar_playlists_2, intersection_2, playlist_means_2 = collaborative_filtering(df_merged, df_track, df_original,
                                                                               pid, 0, 50, 'pearson similarity')

In [289]:
print("Pearson Similarity")
df_similar_tracks_2

Pearson Similarity


,Track name,Artist,No. of Playlists
0,I Write Sins Not Tragedies,Panic! At The Disco,22
1,Centuries,Fall Out Boy,19
2,Welcome to the Black Parade,My Chemical Romance,17
3,My Songs Know What You Did In The Dark (Light ...,Fall Out Boy,17
4,Teenagers,My Chemical Romance,16


In [295]:
df_similar_playlists_2.head()

,Playlist name,pearson similarity
0,🎧🎧🎧,0.994797
1,My Songs,0.993836
2,Good old times,0.993321
3,Jacob,0.992712
4,Summer hits,0.992703


In [252]:
#Evaluation
df_predicted = pd.DataFrame()
for track in intersection_2:
    df_predicted = df_predicted.append(df_merged.loc[df_merged['tid'] == track].head(1))

df_random = pd.DataFrame()
random.seed(500)
for idx in random.sample(range(0, len(df_merged)), 10):
    df_random = df_random.append(df_merged.iloc[idx])
    
df_predicted = df_predicted.drop(['tid', 'pid'], axis=1)
df_random = df_random.drop(['tid', 'pid'], axis=1)

#Scale
scaler = MinMaxScaler()
df_predicted = pd.DataFrame(scaler.fit_transform(df_predicted))
df_random = pd.DataFrame(scaler.fit_transform(df_random))

#Get mean
df_predicted_means = df_predicted.mean()
df_random_means = df_random.mean()

#Similarity measure
print("Pearson similarity of predicted tracks:", 
      np.dot(df_predicted_means, playlist_means_2)/(np.linalg.norm(df_predicted_means)*np.linalg.norm(playlist_means_2)))
print("Pearson similarity of random tracks:", 
      np.dot(df_random_means, playlist_means_2)/(np.linalg.norm(df_random_means)*np.linalg.norm(playlist_means_2)))

Pearson similarity of predicted tracks: 0.90651344956386
Pearson similarity of random tracks: 0.8898942766132006


### Euclidean Distance

In [291]:
df_similar_tracks_3, df_similar_playlists_3, intersection_3, playlist_means_3 = collaborative_filtering(df_merged, df_track, df_original,
                                                                               pid, 0, 50, 'euclidean distance')

In [292]:
print("Euclidean Distance")
df_similar_tracks_3

Euclidean Distance


,Track name,Artist,No. of Playlists
0,I Write Sins Not Tragedies,Panic! At The Disco,23
1,My Songs Know What You Did In The Dark (Light ...,Fall Out Boy,19
2,Centuries,Fall Out Boy,18
3,Welcome to the Black Parade,My Chemical Romance,17
4,Victorious,Panic! At The Disco,16


In [296]:
df_similar_playlists_3.head()

,Playlist name,euclidean distance
0,🎧🎧🎧,0.142414
1,My Songs,0.152821
2,Good old times,0.159219
3,Summer hits,0.165828
4,Alternative/Pop,0.167635


In [241]:
#Evaluation
df_predicted = pd.DataFrame()
for track in intersection_3:
    df_predicted = df_predicted.append(df_merged.loc[df_merged['tid'] == track].head(1))

df_random = pd.DataFrame()
random.seed(50)
for idx in random.sample(range(0, len(df_merged)), 10):
    df_random = df_random.append(df_merged.iloc[idx])
    
df_predicted = df_predicted.drop(['tid', 'pid'], axis=1)
df_random = df_random.drop(['tid', 'pid'], axis=1)

#Scale
scaler = MinMaxScaler()
df_predicted = pd.DataFrame(scaler.fit_transform(df_predicted))
df_random = pd.DataFrame(scaler.fit_transform(df_random))

#Get mean
df_predicted_means = df_predicted.mean()
df_random_means = df_random.mean()

#Similarity measure
print("Euclidean distance of predicted tracks:", np.linalg.norm(df_predicted_means-playlist_means_3))
print("Euclidean distance of random tracks:", np.linalg.norm(df_random_means-playlist_means_3))

Euclidean distance of predicted tracks: 0.747947077909057
Euclidean distance of random tracks: 0.8666165527906632


# Hybrid Filtering

In [18]:
df_track_features = pd.read_csv("lib/tracks_features.csv", index_col=False)
df_track_features = pd.get_dummies(df_track_features, columns=['key'])
df_track_features = df_track_features.drop(['Unnamed: 0', 'pos', 'track_uri', 'artist_uri',
                                            'album_uri', 'duration_ms', 'sparse_id','type', 'id',
                                            'uri', 'track_href', 'analysis_url', 'duration_ms.1',
                                            'time_signature'], axis=1)

In [14]:
def hybrid_filtering(df, df_track, df_features, df_original, df_explode, pid, test_perc, k, metric, n): 

    '''Collaborative'''
    #Remove test set
    pid_indices = df[df['pid']==pid].index
    total_size = len(pid_indices)
    test_size = int(total_size*(test_perc/100))

    test_tracks = []
    for index in pid_indices[:test_size]:
        test_tracks.append(df.loc[index]['tracks'])
        df = df.drop(index)

    #Remove unnecessary columns
    df = df.drop(['tid'], axis=1)


    #Get means
    df_means = df.groupby(['pid']).mean()
    pids = df_means.index.tolist()


    #Scaling
    scaler = MinMaxScaler()
    df_means = pd.DataFrame(scaler.fit_transform(df_means))
    df_means.index = pids
    
    to_scale = ['danceability','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','positive','negative','neutral','compound']
    scaler = MinMaxScaler()
    df_features[to_scale] = scaler.fit_transform(df_features[to_scale])
    

    if metric == 'cosine similarity':
        pid_similars, similarity_values = cosine_sim(df_means, pid, pids, k)
        
        
    elif metric == 'pearson similarity':
        pid_similars, similarity_values = pearson_sim(df_means, pid, pids, k)
        
    elif metric == 'euclidean distance':
        pid_similars, similarity_values = euclidean_dist(df_means, pid, pids, k)

    
    #Get original playlist tracks
    original_playlist = df_original[df_original['pid']==pid].iloc[0]['tracks']

    #Remove overlap with query playlist
    similar_playlists = []
    for pid_similar in pid_similars:
        similar_playlist = df_original[df_original['pid']==pid_similar].iloc[0]['tracks']
        overlap = set(original_playlist).intersection(similar_playlist)
        for track in overlap:
            similar_playlist.remove(track)
        similar_playlists.append(similar_playlist)
        
    #Tracks from similar playlists
    similar_playlists =  [item for sublist in similar_playlists for item in sublist]
    similar_playlists = list(set(similar_playlists))
    
    dfc = pd.DataFrame()
    for track in similar_playlists:
        dfc = dfc.append(df_features.loc[df_features['tid'] == track])
    
    
    '''Content Based'''
    
    other_features = np.array(dfc.iloc[:,4:].values)
    
    song_feature = df_means.iloc[pid].values.reshape(1,-1)
    
    
    if metric == 'cosine similarity':
        dfc['cosine similarity'] = cosine_similarity(other_features, song_feature)
        topn = dfc.sort_values('cosine similarity',ascending = False).head(n+1)[1:][['artist_name', 'track_name', 'album_name','cosine similarity']]


    elif metric == 'euclidean distance':
        dfc['euclidean distance'] = pairwise_distances(other_features, song_feature, metric='euclidean')
        topn = dfc.sort_values('euclidean distance',ascending = True).head(n+1)[1:][['artist_name', 'track_name', 'album_name','euclidean distance']]


    elif metric == 'pearson similarity':
        dfc['pearson similarity']= [pearsonr(song_feature.reshape(-1), x.reshape(-1))[0] for x in other_features]
        topn = dfc.sort_values('pearson similarity',ascending = False).head(n+1)[1:][['artist_name', 'track_name', 'album_name','pearson similarity']]

    
    return topn

### Cosine Similarity

In [15]:
pid = 235
recs_1 = hybrid_filtering(df_merged, df_track, df_track_features, df_original, df, pid, 0, 50, 'cosine similarity',5)    
recs_1

,artist_name,track_name,album_name,cosine similarity
2518,Bowling For Soup,High School Never Ends - Main Version - Explicit,The Great Burrito Extortion Case,0.914495
6348,Nickelback,Something In Your Mouth,Dark Horse,0.913682
2854,Florida Georgia Line,Tip It Back,Here's To The Good Times,0.913468
2475,Jimmy Eat World,The Middle,Bleed American,0.913422
20410,Papa Roach,Kick In The Teeth,Time For Annihilation...On The Record & On The...,0.913028


### Pearson Similarity

In [16]:
recs_2 = hybrid_filtering(df_merged, df_track, df_track_features, df_original, df, pid, 0, 50, 'pearson similarity',5)    
recs_2

,artist_name,track_name,album_name,pearson similarity
2348,Paramore,Still Into You,Paramore,0.859147
366,Macklemore & Ryan Lewis,Can't Hold Us - feat. Ray Dalton,The Heist,0.857801
6348,Nickelback,Something In Your Mouth,Dark Horse,0.856428
20402,Mayday Parade,Get Up,Anywhere But Here,0.855909
2854,Florida Georgia Line,Tip It Back,Here's To The Good Times,0.855734


### Euclidean Distance

In [17]:
recs_3 = hybrid_filtering(df_merged, df_track, df_track_features, df_original, df, pid, 0, 50, 'euclidean distance',5)    
recs_3

,artist_name,track_name,album_name,euclidean distance
15668,McFly,Falling In Love,ACTIVE,1.052640
13894,Linkin Park,What I've Done,Minutes To Midnight,1.054158
20495,Monty Are I,One In A Million,Break Through The Silence,1.058944
22759,My Chemical Romance,Summertime,Danger Days: The True Lives Of The Fabulous Ki...,1.060052
6270,Avenged Sevenfold,Shepherd of Fire,Hail to the King,1.064295
